# 5.3.1 Analysis of Version Characteristics of VRI and DRI

In [1]:
from matplotlib import pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False
shapes = ['o', 'v', '^', '<', '>', 's', 'p', '*', 'h', '+']
colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'darkgreen', 'chocolate']

## Read network data

In [2]:
from experiment.functions import func_get_lab_graph, save_graph
from experiment.lab2._0_config_ import LabSpace, SoftwareConfigs2
import os

Graphs = dict()
Labels = dict()
Types = dict()

DRIs = dict()
VRIs = dict()

for softwareName, (status, language, src_dir) in SoftwareConfigs2.items():
    if status:
        modelDir = os.path.join(LabSpace, softwareName)
        labDir = os.path.join(modelDir, '.lab2')
        
        G, labelDict, _, typeDict = func_get_lab_graph(modelDir, labDir, softwareName)
        Graphs[softwareName] = G
        Labels[softwareName] = labelDict
        Types[softwareName] = typeDict
        
        DRIs[softwareName] = dict(G.nodes.data('DRI'))
        VRIs[softwareName] = dict(G.nodes.data('VRI'))
        
        print(f'{softwareName} read completed')
        

In [3]:
Software1Dict = {
    'Nginx': ['Nginx-1.22.1', 'Nginx-1.21.1', 'Nginx-1.20.0', 'Nginx-1.19.0'],
    'Redis': ['Redis-7.0.12', 'Redis-7.0.11', 'Redis-7.0.10', 'Redis-7.0.9'],
    'Node.js': ['Node.js-20.5.0', 'Node.js-20.4.0', 'Node.js-20.3.0', 'Node.js-20.2.0'],
    'VLC': ['VLC-3.0.17', 'VLC-3.0.16', 'VLC-3.0.15', 'VLC-3.0.14']
}

Softwares = [
    'Nginx-1.22.1',
    'Redis-7.0.12',
    'Node.js-20.5.0',
    'VLC-3.0.17'
]

labs = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']


## Draw different versions of VRI and DRI images

In [4]:
from experiment.functions import func_SortDictByValue, func_std
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os

idx = 0
figVRI, axsVRI = plt.subplots(2, 2, figsize=(15, 9))
plt.subplots_adjust(hspace=0.34, wspace=0.2)

figDRI, axsDRI = plt.subplots(2, 2, figsize=(15, 9))
plt.subplots_adjust(hspace=0.34, wspace=0.2)

for _name in Software1Dict:
    versions = Software1Dict[_name]
    
    axVRI = axsVRI[int(idx / 2), idx % 2]
    axDRI = axsDRI[int(idx / 2), idx % 2]
    
    idx += 1

    i = 0
    count = 100
    
    VRIDf = pd.DataFrame(
        columns=pd.MultiIndex.from_product(
               [versions, ['id', 'val', 'label']]
           )
    )
    DRIDf = pd.DataFrame()
    
    for softwareName in versions:
        G = Graphs[softwareName]
        labelDict = Labels[softwareName]
        print(f'Loading software {softwareName}')
        
        SortedVRI = func_SortDictByValue(VRIs[softwareName])
        VRIIds = list(SortedVRI.keys())[:count]
        VRIVals = list(SortedVRI.values())[:count]
        VRIVals = [round(np.sqrt(10 * n), 2) for n in VRIVals]
        VRILabels = [labelDict[n] for n in VRIIds]
        
        SortedDRI = func_SortDictByValue(DRIs[softwareName])
        DRIIds = list(SortedDRI.keys())[:count]
        DRIVals = list(SortedDRI.values())[:count]
        DRIVals = [round(np.sqrt(10 * n), 2) for n in DRIVals]
        DRILabels = [labelDict[n] for n in DRIIds]
        
        VRIDf[softwareName, 'id'] = VRIIds
        VRIDf[softwareName, 'val'] = VRIVals
        VRIDf[softwareName, 'label'] = VRILabels
        
        DRIDf[softwareName, 'id'] = DRIIds
        DRIDf[softwareName, 'val'] = DRIVals
        DRIDf[softwareName, 'label'] = DRILabels
        
        X = range(count)
        axVRI.plot(X, VRIVals, label=softwareName, marker=shapes[i], color=colors[i])
        axDRI.plot(X, DRIVals, label=softwareName, marker=shapes[i], color=colors[i])
        i += 1
        
        axVRI.set_xlabel('Function', fontdict={'size': 12})
        axVRI.set_ylabel('VRI', fontdict={'size': 12})
        axVRI.set_title(f'{labs[idx]}) {_name}', y=-0.28, fontdict={'family': 'Times New Roman', 'size': 18})
        
        axDRI.set_xlabel('Function', fontdict={'size': 12})
        axDRI.set_ylabel('DRI', fontdict={'size': 12})
        axDRI.set_title(f'{labs[idx]}) {_name}', y=-0.28, fontdict={'family': 'Times New Roman', 'size': 18})
    
    axVRI.legend()
    axDRI.legend()
    VRIDf.to_csv(os.path.join(LabSpace, f'{_name}-vri-version.csv'))
    DRIDf.to_csv(os.path.join(LabSpace, f'{_name}-dri-version.csv'))
    
figVRI.show()    
figDRI.show()    
    

### Record the top 10 nodes ranked in VRI and DRI

In [5]:
import pandas as pd

# figVRI, axsVRI = plt.subplots(2, 2, figsize=(20, 10))
# figDRI, axsDRI = plt.subplots(2, 2, figsize=(20, 10))

idx = 0

dfsVRI = dict()
dfsDRI = dict()
dfsCount = 10

mainVersions = ['Nginx-1.22.1', 'Redis-7.0.12']

for _name in ['Nginx', 'Redis']:
    versions = Software1Dict[_name]
    cols = list(versions)
    cols.insert(0, 'name')
    
    dfsVRI[_name] = pd.DataFrame(columns=cols)
    dfsDRI[_name] = pd.DataFrame(columns=cols)
    
    # axVRI = axsVRI[int(idx / 2), idx % 2]
    # axDRI = axsDRI[int(idx / 2), idx % 2]
    
    labelVRIX = None
    labelDRIX = None
    
    i = 0
    count = 100
    tCount = 30
    
    for softwareName in versions:
        G = Graphs[softwareName]
        labelDict = Labels[softwareName]
        
        print(f'Loading software {softwareName}')
        
        SortedVRI = func_SortDictByValue(VRIs[softwareName])
        VRIIds = list(SortedVRI.keys())[:count]
        VRIVals = list(SortedVRI.values())[:count]
        VRIVals = [round(np.sqrt(10 * n), 2) for n in VRIVals]
        VRILabels = [labelDict[n] for n in VRIIds]
        
        SortedDRI = func_SortDictByValue(DRIs[softwareName])
        DRIIds = list(SortedDRI.keys())[:count]
        DRIVals = list(SortedDRI.values())[:count]
        DRIVals = [round(np.sqrt(10 * n), 2) for n in DRIVals]
        DRILabels = [labelDict[n] for n in DRIIds]
        
        if softwareName in mainVersions:
            _bName = softwareName
            
            labelVRIX = VRILabels[:tCount]
            valVRIY = VRIVals[:tCount]
            dfsVRI[_name]['name'] = labelVRIX[:dfsCount]
            vals = []
            for j in range(dfsCount):
                vals.append(f'{j+1}/{valVRIY[j]}')
            dfsVRI[_name][softwareName] = vals
    
            labelDRIX = DRILabels[:tCount]
            valDRIY = DRIVals[:tCount]
            dfsDRI[_name]['name'] = labelDRIX[:dfsCount]
            vals = []
            for j in range(dfsCount):
                vals.append(f'{j+1}/{valDRIY[j]}')
            dfsDRI[_name][softwareName] = vals
        else:
            valVRIY = []
            j = 0
            vals = []
            for label in labelVRIX:
                tIdx = -1
                lb = None
                if label in VRILabels:
                    tIdx = VRILabels.index(label)
                    lb = VRIVals[tIdx]
                valVRIY.append(lb)
                if j < 10:
                    vals.append(f'{tIdx+1}/{lb}')
                j+=1
            dfsVRI[_name][softwareName] = vals
                
            valDRIY = []
            j = 0
            vals = []
            for label in labelDRIX:
                tIdx = -1
                lb = None
                
                if label in DRILabels:
                    tIdx = DRILabels.index(label)
                    lb = DRIVals[tIdx]
                valDRIY.append(lb)
                if j < 10:
                    vals.append(f'{tIdx+1}/{lb}')
                j+=1
            dfsDRI[_name][softwareName] = vals
        
        # axVRI.scatter(range(1, tCount+1), valVRIY, label=softwareName, marker=shapes[i], color=colors[i])
        # axDRI.plot(range(1, tCount+1), valDRIY, label=softwareName, marker=shapes[i], color=colors[i])
        i += 1
        
    # axVRI.set_title(f'Based on {_fName}')
    # axDRI.set_title(f'Based on {_fName}')
    # axVRI.legend()
    # axDRI.legend()
    
    idx += 1    

In [6]:
dfsVRI['Nginx'].to_csv(os.path.join(LabSpace, 'Nginx.vri.csv'))
dfsDRI['Nginx'].to_csv(os.path.join(LabSpace, 'Nginx.dri.csv'))
dfsVRI['Redis'].to_csv(os.path.join(LabSpace, 'Redis.vri.csv'))
dfsDRI['Redis'].to_csv(os.path.join(LabSpace, 'Redis.dri.csv'))


In [7]:
dfsVRI['Nginx']

In [8]:
dfsDRI['Redis']